### Example

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from draw_cml_map import *

name_map = 'TRY_MAP'
h = draw_cml_map(out_path='/dir/to/output/',
            data_path='/dir/to/data/',
            rawdata_dir='rawdata',
            metadata_file_name='export.csv',
            list_of_link_id_to_drop=['4673-7HZ4','7HZ4-4673'],
            list_of_link_id_to_color=['TS01-7330','j220-s220'],
            color_of_specific_links = 'red',
            name_of_map_file=name_map,
            distort_lat_lon=True)

Carriers:
['Cellcom']
Number of links in map: 
12
/Users/adameshel/Documents/Python_scripts/I_Change/try/TRY_MAP.html
Map under the name TRY_MAP.html was generated.


### Draw on same map additional CMLs in different colors and add a grid

In [11]:
h = draw_cml_map(out_path='/dir/to/output/',
            data_path='/dir/to/data/',
            metadata_file_name='pelephone jan 2 2020.csv',
            handle=h,
            area_min_lon=35.0,
            area_max_lon=36.0,
            area_min_lat=31.5,
            area_max_lat=32.1,
            color_of_links='blue',
            num_of_gridlines=30,
            name_of_map_file=name_map)

Carriers:
['Pelephone']
Number of links in map: 
62
/Users/adameshel/Documents/Python_scripts/I_Change/try/TRY_MAP.html
Map under the name TRY_MAP.html was generated.
